In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt

import json
import enum
import os

from col_mnist import ColMNIST
from three_d_shapes_ds import ThreeDShapes
from models import compute_layer_blocks_out



In [2]:
%load_ext autoreload
%autoreload 2
%pdb off

Automatic pdb calling has been turned OFF


In [5]:
from three_d_shapes_ds import ThreeDShapes
trainloader = torch.utils.data.DataLoader(
                                      ThreeDShapes(filename='data/3dshapes.h5',
                                                   transform=torchvision.transforms.Compose([
                                                       torchvision.transforms.ToPILImage(), 
                                                       torchvision.transforms.Resize((32, 32)),
                                                       torchvision.transforms.ToTensor(),
                                      ]), filtered = False, dt_labels = True),
                                      batch_size=32, shuffle=True)

<class 'numpy.ndarray'>


RuntimeError: CUDA out of memory. Tried to allocate 16.48 GiB (GPU 0; 39.59 GiB total capacity; 16.48 GiB already allocated; 3.22 GiB free; 16.48 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

start_time = time.time()
for i, d in enumerate(trainloader):
    data, (latents, target) = d
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
def target_vec_to_class(vec):
    labels = torch.zeros((vec.shape[0]))
    for i, v in enumerate(vec):
        if v[2] > 0.5: #object hue
            if v[0] > 0.5: #floor hue
                if v[5] > 0: #orientation
                    labels[i] = 7 #object of cooler color, cooler floor, orientation?
                else: #orientation
                    labels[i] = 6 #object of cooler color, cooler floor, orientation?
            else: #floor hue
                if v[4] > 1: #shape 
                    labels[i] = 5 #object of cooler color, warmer floor, pill or sphere
                else: #shape
                    labels[i] = 4 #object of cooler color, warmer floor, cylinder or cube
        else: #object hue
            if v[1] > 0.5: #wall hue
                if v[3] > 1: #scale
                    labels[i] = 3 #object of warmer color, cooler walls, bigger scale
                else: #scale
                    labels[i] = 2 #object of warmer color, cooler walls, smaller scale
            else:
                if v[4] > 2: #shape
                    labels[i] = 1 #object of warmer color, warmer walls, pill
                else: #shape
                    labels[i] = 0 #object of warmer color, warmer walls, sphere, cylinder or cube                   

    return labels.long()

print(target_vec_to_class(latents)==target)

In [ ]:
print(target_vec_to_class(latents))
print(target)